In [6]:
1. https://www.kmdb.or.kr/db/have/detailSearch/imageSearch 방문
2. /html/body/div[2]/div[5]/div[2]/section/div/div[3]/div[3]/span/a 클릭(더보기 버튼)
3. startswith('http://file.koreafilm.or.kr/poster/') 파일 클릭(이미지)
4. /html/body/div[4]/div[2]/div[1]/a 클릭(x 버튼)
5. /html/body/div[2]/div[5]/div[2]/form/section/div[2]/div[1]/div[1]/div/div[2]/a 클릭(기본정보)
6. mList12 클래스 안의 mTag1 gray 클래스 안의 텍스트를 리스트로 수집하며 프린트(장르)
7. genre_mapping의 정보에 따라 라벨링하기, 복합 장르일 경우 or 연산을 통해 라벨 부여하고 수집하며 프린트(장르 라벨링)
8. /html/body/div[2]/div[5]/div[2]/div[1]/div/div/a 클릭(뒤로 가기), https://www.kmdb.or.kr/db/have/detailSearch/imageSearch 페이지로 돌아올 때까지 클릭 수행
9. 방금 클릭했던 startswith('http://file.koreafilm.or.kr/poster/') 이미지를 G:/내 드라이브/project_poster/genre_code_movie_posters에 저장하며 파일명은 7번 과정을 통해 얻은 라벨로 하기
10. 다음 startswith('http://file.koreafilm.or.kr/poster/') 클릭, 4번부터 9번까지를 한 페이지 내에서 12번 수행
11.
buttons_xpath = [
    "/html/body/div[2]/div[5]/div[2]/section/div/div[3]/div[3]/div/a[1]",
    "/html/body/div[2]/div[5]/div[2]/section/div/div[3]/div[3]/div/a[2]",
    "/html/body/div[2]/div[5]/div[2]/section/div/div[3]/div[3]/div/a[3]",
    "/html/body/div[2]/div[5]/div[2]/section/div/div[3]/div[3]/div/a[4]",
    "/html/body/div[2]/div[5]/div[2]/section/div/div[3]/div[3]/div/span[2]/a[1]"
]
를 누르며 페이지 넘기기 진행
12. 각 페이지마다 10번에서 말했듯이 2번부터 9번까지를 12번 수행

import time
import re
import os
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup 

# 장르 정보를 매핑하기 위한 딕셔너리 (비트 문자열)
genre_mapping = {
    "드라마": "1000000000000000000000000000000000000000000",
    "첩보": "0100000000000000000000000000000000000000000",
    "옴니버스": "0010000000000000000000000000000000000000000",
    "뮤직": "0001000000000000000000000000000000000000000",
    "동성애": "0000100000000000000000000000000000000000000",
    "로드무비": "0000010000000000000000000000000000000000000",
    "아동": "0000001000000000000000000000000000000000000",
    "하이틴(고교)": "0000000100000000000000000000000000000000000",
    "전기": "0000000010000000000000000000000000000000000",
    "청춘영화": "0000000001000000000000000000000000000000000",
    "재난": "0000000000100000000000000000000000000000000",
    "문예": "0000000000010000000000000000000000000000000",
    "연쇄극": "0000000000001000000000000000000000000000000",
    "신파": "0000000000000100000000000000000000000000000",
    "활극": "0000000000000010000000000000000000000000000",
    "반공/분단": "0000000000000001000000000000000000000000000",
    "군사": "0000000000000000100000000000000000000000000",
    "계몽": "0000000000000000010000000000000000000000000",
    "사회물(경향)": "0000000000000000001000000000000000000000000",
    "스포츠": "0000000000000000000100000000000000000000000",
    "합작(번안물)": "0000000000000000000010000000000000000000000",
    "종교": "0000000000000000000001000000000000000000000",
    "무협": "0000000000000000000000100000000000000000000",
    "미스터리": "0000000000000000000000010000000000000000000",
    "SF": "0000000000000000000000001000000000000000000",
    "코메디": "0000000000000000000000000100000000000000000",
    "느와르": "0000000000000000000000000010000000000000000",
    "액션": "0000000000000000000000000001000000000000000",
    "범죄": "0000000000000000000000000000100000000000000",
    "어드벤처": "0000000000000000000000000000010000000000000",
    "가족": "0000000000000000000000000000001000000000000",
    "에로": "0000000000000000000000000000000100000000000",
    "멜로/로맨스": "0000000000000000000000000000000010000000000",
    "공포": "0000000000000000000000000000000001000000000",
    "뮤지컬": "0000000000000000000000000000000000100000000",
    "시대극/사극": "0000000000000000000000000000000000010000000",
}

# Selenium 드라이버 설정
driver = webdriver.Chrome()

# 1. URL 방문
page_url = 'https://www.kmdb.or.kr/db/have/detailSearch/imageSearch'
driver.get(page_url)
time.sleep(1)

# 2. 첫 번째 버튼 클릭 (필요에 따라)
try:
    first_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div[5]/div[2]/section/div/div[3]/div[3]/span/a"))
    )
    first_button.click()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[contains(@style, 'background-image')]")))
    print("첫 번째 버튼 클릭 성공")
except Exception as e:
    print(f"첫 번째 버튼 클릭 중 오류 발생: {e}")
    driver.quit()
    
# 이미지 넘버링 및 대기 시간 설정
image_number = 0
sleep_time = 1
save_path = r'G:/내 드라이브/project_poster/genre_code_movie_posters'
if not os.path.exists(save_path):
    os.makedirs(save_path)

# 중복 방지를 위한 세트
selected_images = set()


# 3. 이미지 크롤링 함수
def crawl_images(batch_size=100):
    global total_images, selected_images

    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        posters = soup.find_all('span', style=re.compile(r'background-image'))
    except Exception as e:
        print(f"이미지를 찾는 중 오류 발생: {e}")
        return
    if not posters:
        print("No posters found on this page.")

        for span in posters:
            try:
                # 이미지를 새로 가져옴 (stale element 방지)
                style = span.get_attribute('style')
                img_url_match = re.search(r'url\((.*?)\)', style)

                if img_url_match:
                    img_url = img_url_match.group(1).strip("'\"")

                    # 중복 방지: 이미 처리한 이미지 URL은 스킵
                    if img_url in selected_images:
                        continue

                    # 새 이미지 URL만 처리
                    if img_url.startswith('http://file.koreafilm.or.kr/poster/'):
                        print(f"Found image URL: {img_url}")
                        selected_images.add(img_url)  # 중복 방지를 위해 기록

                        # 이미지 클릭 및 장르 정보 수집
                        try:
                            span.click()
                            time.sleep(sleep_time)  # 페이지 로딩 대기

                            # x 버튼 클릭
                            x_button = driver.find_element(By.XPATH, "/html/body/div[4]/div[2]/div[1]/a")
                            x_button.click()
                            time.sleep(sleep_time)  # 클릭 후 대기

                            # 장르 버튼 클릭
                            genre_button = WebDriverWait(driver, 10).until(
                                EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div[5]/div[2]/form/section/div[2]/div[1]/div[1]/div/div[2]/a"))
                            )
                            genre_button.click()
                            time.sleep(sleep_time)

                            # 장르 텍스트 수집
                            genre_element = WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div[5]/div[2]/form/section/div[5]/div[4]/dl/dd[1]"))
                            )
                            genre_text = genre_element.text
                            genres = [g.strip() for g in genre_text.split('#') if g]
                            print(f"Collected genres: {genres}")
                            
                            # 장르를 이진수로 변환
                            genre_code = 0
                            for genre in genres:
                                genre_code |= genre_mapping.get(genre, 0)

                            print(f"Genre collected: {genres}, Genre code: {bin(genre_code)}")

                            # 이미지 저장
                            filename = f"{bin(genre_code)[2:]}_{image_number}.jpg"
                            full_path = os.path.join(save_path, filename)

                            response = requests.get(img_url, stream=True)
                            if response.status_code == 200:
                                with open(full_path, 'wb') as handler:
                                    handler.write(response.content)
                                print(f"Saved image as {full_path}")
                                image_number += 1
                            else:
                                print(f"Failed to download image: {response.status_code}")
                                
                        except Exception as e:
                            print(f"")
                            driver.back()  # 오류가 발생하면 페이지를 되돌리기
            except Exception as e:
                print(f"")
                                
        # 5. 장르 페이지로 이동 후 다시 리스트로 돌아가기
        max_back_attempts = 10  # 최대 뒤로 가기 시도 횟수
        back_attempts = 0

        while back_attempts < max_back_attempts:
            try:
                # 뒤로 가기 버튼 클릭
                back_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div[5]/div[2]/div[1]/div/div/a"))  # 뒤로 가기 버튼의 XPATH
                )
                back_button.click()
                time.sleep(2)  # 버튼 클릭 후 페이지 로딩 대기

                # 특정 URL의 일부가 포함되었는지 확인
                WebDriverWait(driver, 10).until(
                    EC.url_contains('imageSearch')  # 돌아와야 할 URL의 일부
                )

                # URL이 확인되면 반복 종료
                print("페이지가 성공적으로 돌아왔습니다.")
                break  # 페이지가 돌아오면 while 루프 종료

            except Exception as e:
                print(f"뒤로 가기 버튼 클릭 중 오류 발생: {e}")
                back_attempts += 1
                if back_attempts >= max_back_attempts:
                    print("최대 뒤로 가기 시도 횟수 초과. 반복을 중단합니다.")
                    driver.quit()
                    
    return images_downloaded

# 드라이버 종료
driver.quit()

이미지 처리 중 오류 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.60)
Stacktrace:
	GetHandleVerifier [0x00007FF65D24B125+29573]
	(No symbol) [0x00007FF65D1BFF50]
	(No symbol) [0x00007FF65D07B6EA]
	(No symbol) [0x00007FF65D04FCD5]
	(No symbol) [0x00007FF65D0FEF67]
	(No symbol) [0x00007FF65D117FC1]
	(No symbol) [0x00007FF65D0F70A3]
	(No symbol) [0x00007FF65D0C12DF]
	(No symbol) [0x00007FF65D0C2441]
	GetHandleVerifier [0x00007FF65D57C76D+3377613]
	GetHandleVerifier [0x00007FF65D5C7B67+3685831]
	GetHandleVerifier [0x00007FF65D5BCF8B+3641835]
	GetHandleVerifier [0x00007FF65D30B2A6+816390]
	(No symbol) [0x00007FF65D1CB25F]
	(No symbol) [0x00007FF65D1C7084]
	(No symbol) [0x00007FF65D1C7220]
	(No symbol) [0x00007FF65D1B607F]
	BaseThreadInitThunk [0x00007FFD4DD6257D+29]
	RtlUserThreadStart [0x00007FFD4EBEAF28+40]

이미지 처리 중 오류 발생: Message: no such window: target window already closed
from unknown error: web view not fo